In [ ]:
# 런타임 재시작시
from google.colab import drive
drive.mount('/content/drive')

import os, torch

os.chdir("/content/drive/MyDrive/CProject/Tank-Model-Classification")
print("경로:", os.getcwd())

!git status

print("CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Mounted at /content/drive
경로: /content/drive/MyDrive/CProject/Tank-Model-Classification
Refresh index: 100% (41/41), done.
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    tank_nontank.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	tank_nontank_yolo.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
CUDA: False


In [26]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	deleted:    tank_nontank.ipynb
	new file:   tank_nontank_yolo.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	tank_nontank_resnet_efficientnet.ipynb



In [27]:
!git checkout main

Switched to branch 'main'
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)


In [17]:
!git config --global user.email "gpwnsla1215@naver.com"
!git config --global user.name "hizoo66"

In [14]:
!git merge main


Updating 4d80147..a37cd09
Updating files: 100% (39/39), done.
Fast-forward
 .gitignore                                        |   1 +
 runs/detect/train/BoxF1_curve.png                 | Bin 0 -> 223117 bytes
 runs/detect/train/BoxPR_curve.png                 | Bin 0 -> 163130 bytes
 runs/detect/train/BoxP_curve.png                  | Bin 0 -> 173148 bytes
 runs/detect/train/BoxR_curve.png                  | Bin 0 -> 193943 bytes
 runs/detect/train/args.yaml                       | 108 ++++++++++++++++++++++
 runs/detect/train/confusion_matrix.png            | Bin 0 -> 171260 bytes
 runs/detect/train/confusion_matrix_normalized.png | Bin 0 -> 190234 bytes
 runs/detect/train/labels.jpg                      | Bin 0 -> 238347 bytes
 runs/detect/train/results.csv                     |  51 ++++++++++
 runs/detect/train/results.png                     | Bin 0 -> 258746 bytes
 runs/detect/train/train_batch0.jpg                | Bin 0 -> 591301 bytes
 runs/detect/train/train_batch1.jpg        

In [32]:
!git fetch origin
!git reset --hard origin/main


Updating files: 100% (42/42), done.
HEAD is now at a428fe3 clean_data_yolo_modelv1


In [30]:
!git log --oneline --decorate


a37cd09 (HEAD -> main) initial_compare_models
1d261f4 faster-rnn-train
a428fe3 (origin/main, origin/HEAD, master) clean_data_yolo_modelv1
4d80147 (origin/master) initial-commit


In [22]:
!git push origin master

Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.25 MiB | 3.85 MiB/s, done.
Total 6 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 2 local objects.
remote: error: GH013: Repository rule violations found for refs/heads/master.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— GitHub Personal Access Token ——————————————————————
remote:        location

### Resnet/ EfficientNet 모델 비교
YOLO → 바운딩박스 crop 데이터 셋 구성
1. 데이터 셋 구조

In [ ]:
### Resnet/ EfficientNet 모델 비교
YOLO → 바운딩박스 crop 데이터 셋 구성
1. 데이터 셋 구조